# Estudo de Caso de Tradução Automática - Implementação da Seção 9.5

Este notebook implementa o estudo de caso de tradução automática da seção 9.5 do livro "Dive into Deep Learning". Vamos focar na tradução de Inglês para Francês utilizando o conjunto de dados Tatoeba.

Como estamos implementando isso do zero, vamos definir todas as utilidades necessárias sem depender do pacote d2l.

# Tradução Automática com Redes Neurais

Este notebook implementa o estudo de caso de tradução automática da seção 9.5 do livro Dive into Deep Learning.

Vamos focar na tradução de Inglês para Francês utilizando o conjunto de dados Tatoeba.

## 9.5.1. Download e Pré-processamento do Conjunto de Dados

Primeiro, vamos baixar o conjunto de dados Inglês-Francês do Projeto Tatoeba.

In [1]:
import os
import torch
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import requests
import zipfile
from io import BytesIO
import re
from collections import Counter

In [ ]:
# Definir constantes e funções utilitárias
DATA_URL = 'http://d2l-data.s3-accelerate.amazonaws.com/'
TATOEBA_URL = DATA_URL + 'fra-eng.zip'

def download_extract(url, target_dir='data'):
    """Baixa e extrai um arquivo zip."""
    # Cria o diretório de destino se não existir
    os.makedirs(target_dir, exist_ok=True)
    
    # Extrai o nome do arquivo da URL
    fname = url.split('/')[-1]
    data_dir = os.path.join(target_dir, fname.split('.')[0])
    
    # Retorna se o diretório de dados já existir
    if os.path.exists(data_dir):
        return data_dir
    
    # Baixa o arquivo
    print(f"Baixando {fname} de {url}...")
    r = requests.get(url)
    
    # Extrai o arquivo zip
    with zipfile.ZipFile(BytesIO(r.content)) as zf:
        zf.extractall(target_dir)
    
    return data_dir

def read_data_nmt():
    """Carrega o conjunto de dados Inglês-Francês."""
    data_dir = download_extract(TATOEBA_URL)
    with open(os.path.join(data_dir, 'fra.txt'), 'r', encoding='utf-8') as f:
        return f.read()

raw_text = read_data_nmt()
print(raw_text[:75])

Go.	Va !
Hi.	Salut !
Run!	Cours !
Run!	Courez !
Who?	Qui ?
Wow!	Ça alors !

Go.	Va !
Hi.	Salut !
Run!	Cours !
Run!	Courez !
Who?	Qui ?
Wow!	Ça alors !



Após baixar o conjunto de dados, continuamos com as etapas de pré-processamento para os dados de texto bruto. Substituímos espaços não-quebráveis por espaços regulares, convertemos letras maiúsculas em minúsculas e inserimos espaços entre palavras e sinais de pontuação.

In [ ]:
def preprocess_nmt(text):
    """Pré-processa o conjunto de dados Inglês-Francês."""
    def no_space(char, prev_char):
        return char in set(',.!?') and prev_char != ' '

    # Substitui espaços não-quebráveis por espaços normais e converte letras maiúsculas
    # para minúsculas
    text = text.replace('\u202f', ' ').replace('\xa0', ' ').lower()
    # Insere espaço entre palavras e sinais de pontuação
    out = [' ' + char if i > 0 and no_space(char, text[i - 1]) else char
           for i, char in enumerate(text)]
    return ''.join(out)

text = preprocess_nmt(raw_text)
print(text[:80])

## 9.5.2. Tokenização

Para tradução automática, preferimos tokenização em nível de palavra em vez de tokenização em nível de caractere. A função a seguir tokeniza os primeiros `num_examples` pares de sequências de texto, onde cada token é uma palavra ou um sinal de pontuação.

In [ ]:
def tokenize_nmt(text, num_examples=None):
    """Tokeniza o conjunto de dados Inglês-Francês."""
    source, target = [], []
    for i, line in enumerate(text.split('\n')):
        if num_examples and i > num_examples:
            break
        parts = line.split('\t')
        if len(parts) == 2:
            source.append(parts[0].split(' '))
            target.append(parts[1].split(' '))
    return source, target

source, target = tokenize_nmt(text)
print(source[:6])
print(target[:6])

Vamos criar um histograma do número de tokens por sequência de texto. Neste conjunto de dados simples de Inglês-Francês, a maioria das sequências de texto tem menos de 20 tokens.

In [ ]:
# Configura o tamanho da figura
plt.figure(figsize=(10, 6))

# Cria o histograma
source_lengths = [len(l) for l in source]
target_lengths = [len(l) for l in target]

plt.hist([source_lengths, target_lengths], bins=20, label=['origem', 'destino'])
plt.legend(loc='upper right')
plt.title('Histograma de Comprimentos de Sequência')
plt.xlabel('Comprimento')
plt.ylabel('Contagem')
plt.show()

# Imprime algumas estatísticas
print(f"Comprimento máximo da origem: {max(source_lengths)}")
print(f"Comprimento máximo do destino: {max(target_lengths)}")
print(f"Comprimento médio da origem: {sum(source_lengths)/len(source_lengths):.2f}")
print(f"Comprimento médio do destino: {sum(target_lengths)/len(target_lengths):.2f}")

## 9.5.3. Vocabulário

Como nosso conjunto de dados de tradução automática consiste em pares de idiomas, precisamos construir dois vocabulários, um para o idioma de origem (Inglês) e outro para o idioma de destino (Francês). Com a tokenização em nível de palavra, o tamanho do vocabulário será significativamente maior do que usando tokenização em nível de caractere.

In [ ]:
class Vocab:
    """Vocabulário para texto."""
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        if tokens is None:
            tokens = []
        if reserved_tokens is None:
            reserved_tokens = []
        
        # Conta frequências de tokens
        counter = Counter([token for line in tokens for token in line])
        self.token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True)
        
        # Cria mapeamento token para índice
        self.idx_to_token = ['<unk>'] + reserved_tokens
        self.token_to_idx = {token: idx for idx, token in enumerate(self.idx_to_token)}
        
        # Adiciona tokens que atendem ao limiar de frequência
        for token, freq in self.token_freqs:
            if freq >= min_freq and token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1
    
    def __len__(self):
        return len(self.idx_to_token)
    
    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.token_to_idx['<unk>'])
        return [self.__getitem__(token) for token in tokens]
    
    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

# Cria vocabulário de origem
src_vocab = Vocab(source, min_freq=2, reserved_tokens=['<pad>', '<bos>', '<eos>'])
print(f"Tamanho do vocabulário de origem: {len(src_vocab)}")

In [ ]:
# Cria vocabulário de destino
tgt_vocab = Vocab(target, min_freq=2, reserved_tokens=['<pad>', '<bos>', '<eos>'])
print(f"Tamanho do vocabulário de destino: {len(tgt_vocab)}")

## 9.5.4. Carregando o Conjunto de Dados

Para eficiência computacional, processamos minilotes de sequências com o mesmo comprimento através de truncamento e preenchimento. Se uma sequência tiver menos que `num_steps` tokens, preenchemos com o token `<pad>`. Se tiver mais que `num_steps` tokens, truncamos para manter apenas os primeiros `num_steps` tokens.

In [ ]:
def truncate_pad(line, num_steps, padding_token):
    """Trunca ou preenche sequências."""
    if len(line) > num_steps:
        return line[:num_steps]  # Trunca
    return line + [padding_token] * (num_steps - len(line))  # Preenche

# Exemplo de truncamento e preenchimento
sample_line = src_vocab[source[0]]  # Converte tokens para índices
padded_line = truncate_pad(sample_line, 10, src_vocab['<pad>'])
print(f"Linha original: {source[0]}")
print(f"Linha indexada: {sample_line}")
print(f"Linha preenchida (tamanho 10): {padded_line}")

Agora definimos uma função para transformar sequências de texto em minilotes para treinamento. Adicionamos o token especial `<eos>` ao final de cada sequência para indicar o fim da sequência.

In [ ]:
def build_array_nmt(lines, vocab, num_steps):
    """Transforma sequências de texto de tradução automática em minilotes."""
    lines = [vocab[l] for l in lines]
    lines = [l + [vocab['<eos>']] for l in lines]
    array = torch.tensor([truncate_pad(
        l, num_steps, vocab['<pad>']) for l in lines])
    valid_len = (array != vocab['<pad>']).sum(1)
    return array, valid_len

# Exemplo de construção de arrays
src_array, src_valid_len = build_array_nmt(source[:3], src_vocab, 10)
print("Array de origem:")
print(src_array)
print("\nComprimentos válidos:")
print(src_valid_len)

## 9.5.5. Juntando Tudo

Finalmente, definimos a função `load_data_nmt` para retornar o iterador de dados junto com os vocabulários de origem e destino.

In [ ]:
def load_data_nmt(batch_size, num_steps, num_examples=600):
    """Retorna o iterador e os vocabulários do conjunto de dados de tradução."""
    text = preprocess_nmt(read_data_nmt())
    source, target = tokenize_nmt(text, num_examples)
    src_vocab = Vocab(source, min_freq=2,
                     reserved_tokens=['<pad>', '<bos>', '<eos>'])
    tgt_vocab = Vocab(target, min_freq=2,
                     reserved_tokens=['<pad>', '<bos>', '<eos>'])
    src_array, src_valid_len = build_array_nmt(source, src_vocab, num_steps)
    tgt_array, tgt_valid_len = build_array_nmt(target, tgt_vocab, num_steps)
    
    # Cria o DataLoader do PyTorch
    dataset = torch.utils.data.TensorDataset(
        src_array, src_valid_len, tgt_array, tgt_valid_len)
    data_iter = torch.utils.data.DataLoader(
        dataset, batch_size, shuffle=True)
    
    return data_iter, src_vocab, tgt_vocab

# Cria um pequeno iterador de dados
train_iter, src_vocab, tgt_vocab = load_data_nmt(batch_size=2, num_steps=8)

# Examina o primeiro lote
for X, X_valid_len, Y, Y_valid_len in train_iter:
    print('X:', X)
    print('Comprimentos válidos para X:', X_valid_len)
    print('Y:', Y)
    print('Comprimentos válidos para Y:', Y_valid_len)
    break

Vamos testar nosso carregamento de dados lendo o primeiro minilote do conjunto de dados Inglês-Francês:

In [ ]:
# Classe base do codificador
class Encoder(nn.Module):
    """A interface base do codificador para a arquitetura codificador-decodificador."""
    def __init__(self, **kwargs):
        super(Encoder, self).__init__(**kwargs)

    def forward(self, X, *args):
        raise NotImplementedError

# Classe base do decodificador
class Decoder(nn.Module):
    """A interface base do decodificador para a arquitetura codificador-decodificador."""
    def __init__(self, **kwargs):
        super(Decoder, self).__init__(**kwargs)

    def init_state(self, enc_outputs, *args):
        raise NotImplementedError

    def forward(self, X, state):
        raise NotImplementedError

# Arquitetura Codificador-Decodificador
class EncoderDecoder(nn.Module):
    """A classe base para a arquitetura codificador-decodificador."""
    def __init__(self, encoder, decoder, **kwargs):
        super(EncoderDecoder, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, enc_X, dec_X, *args):
        enc_outputs = self.encoder(enc_X, *args)
        dec_state = self.decoder.init_state(enc_outputs, *args)
        return self.decoder(dec_X, dec_state)

## 9.5.6. Implementação do Modelo Sequência-para-Sequência

Agora vamos implementar um modelo sequência-para-sequência (seq2seq) com uma arquitetura codificador-decodificador para nossa tarefa de tradução automática.

In [ ]:
class Seq2SeqEncoder(Encoder):
    """O codificador RNN para aprendizagem sequência-para-sequência."""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super(Seq2SeqEncoder, self).__init__(**kwargs)
        # Camada de embedding
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size, num_hiddens, num_layers,
                          dropout=dropout)

    def forward(self, X, *args):
        # X formato: (batch_size, seq_len)
        # Primeiro, converte X para formato: (seq_len, batch_size) para RNN
        X = X.T
        # Converte de índices de token para embeddings
        X = self.embedding(X)  # formato: (seq_len, batch_size, embed_size)
        # A saída `X` formato: (seq_len, batch_size, num_hiddens)
        # `state` formato: (num_layers, batch_size, num_hiddens)
        output, state = self.rnn(X)
        # `output` formato: (seq_len, batch_size, num_hiddens)
        # `state` formato: (num_layers, batch_size, num_hiddens)
        return output, state

In [ ]:
# Teste do codificador
encoder = Seq2SeqEncoder(vocab_size=len(src_vocab), embed_size=8, num_hiddens=16,
                      num_layers=2, dropout=0.1)
batch_size, seq_len = 4, 7
X = torch.ones((batch_size, seq_len), dtype=torch.long)
output, state = encoder(X)
print(f"Formato da saída do codificador: {output.shape}")
print(f"Formato do estado do codificador: {state.shape}")

In [ ]:
class Seq2SeqDecoder(Decoder):
    """O decodificador RNN para aprendizagem sequência-para-sequência."""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                dropout=0, **kwargs):
        super(Seq2SeqDecoder, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.GRU(embed_size + num_hiddens, num_hiddens, num_layers,
                          dropout=dropout)
        self.dense = nn.Linear(num_hiddens, vocab_size)

    def init_state(self, enc_outputs, *args):
        # Usa o estado final do codificador como estado inicial do decodificador
        return enc_outputs[1]

    def forward(self, X, state):
        # X formato: (batch_size, seq_len)
        # Primeiro, converte X para formato: (seq_len, batch_size) para RNN
        X = X.T
        # Obtém o último estado escondido do estado do codificador
        # Transmite o contexto para (seq_len, batch_size, num_hiddens)
        context = state[-1].repeat(X.shape[0], 1, 1)
        # Incorpora a entrada
        X = self.embedding(X)  # (seq_len, batch_size, embed_size)
        # Concatena o contexto e os embeddings
        X_and_context = torch.cat((X, context), 2)
        # Calcula as saídas do decodificador
        output, state = self.rnn(X_and_context, state)
        # Aplica a camada linear final
        output = self.dense(output).permute(1, 0, 2)
        # `output` formato: (batch_size, seq_len, vocab_size)
        # `state` formato: (num_layers, batch_size, num_hiddens)
        return output, state

### Codificador RNN

Agora vamos implementar um codificador RNN para aprendizagem sequência-para-sequência.

In [ ]:
# Teste do decodificador
decoder = Seq2SeqDecoder(vocab_size=len(tgt_vocab), embed_size=8, num_hiddens=16,
                      num_layers=2, dropout=0.1)
state = encoder(X)[1]
output, state = decoder(X, state)
print(f"Formato da saída do decodificador: {output.shape}")
print(f"Formato do estado do decodificador: {state.shape}")

Vamos testar o codificador:

In [ ]:
def sequence_mask(X, valid_len, value=0):
    """Mascara entradas irrelevantes em sequências."""
    maxlen = X.size(1)
    mask = torch.arange((maxlen), dtype=torch.float32,
                      device=X.device)[None, :] < valid_len[:, None]
    X[~mask] = value
    return X

class MaskedSoftmaxCELoss(nn.CrossEntropyLoss):
    """A perda de entropia cruzada softmax com máscaras."""
    # `pred` formato: (batch_size, seq_len, vocab_size)
    # `label` formato: (batch_size, seq_len)
    # `valid_len` formato: (batch_size,)
    def forward(self, pred, label, valid_len):
        weights = torch.ones_like(label)
        weights = sequence_mask(weights, valid_len)
        self.reduction = 'none'
        unweighted_loss = super(MaskedSoftmaxCELoss, self).forward(
            pred.permute(0, 2, 1), label)
        weighted_loss = (unweighted_loss * weights).mean(dim=1)
        return weighted_loss

### Decodificador RNN

Agora vamos implementar um decodificador RNN para aprendizagem sequência-para-sequência.

In [ ]:
# Função de recorte de gradiente
def grad_clipping(model, theta):
    """Recorta o gradiente."""
    if isinstance(model, nn.Module):
        params = [p for p in model.parameters() if p.requires_grad]
    else:
        params = model.params
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

# Função de treinamento
def train_seq2seq(model, data_iter, lr, num_epochs, device):
    """Treina um modelo seq2seq."""
    def xavier_init_weights(m):
        if type(m) == nn.Linear:
            nn.init.xavier_uniform_(m.weight)
        if type(m) == nn.GRU:
            for param in m._flat_weights_names:
                if "weight" in param:
                    nn.init.xavier_uniform_(m._parameters[param])
    
    model.apply(xavier_init_weights)
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn = MaskedSoftmaxCELoss()
    model.train()
    
    for epoch in range(num_epochs):
        total_loss = 0
        num_batches = 0
        
        for batch in data_iter:
            optimizer.zero_grad()
            X, X_valid_len, Y, Y_valid_len = [x.to(device) for x in batch]
            bos = torch.tensor([tgt_vocab['<bos>']] * Y.shape[0],
                              device=device).reshape(-1, 1)
            dec_input = torch.cat([bos, Y[:, :-1]], 1)  # Teacher forcing
            Y_hat, _ = model(X, dec_input)
            loss = loss_fn(Y_hat, Y, Y_valid_len)
            loss.sum().backward()  # Torna a perda escalar para backward()
            grad_clipping(model, 1)
            optimizer.step()
            
            total_loss += loss.sum().item()
            num_batches += 1
        
        avg_loss = total_loss / num_batches
        if (epoch + 1) % 10 == 0:
            print(f'época {epoch + 1}, perda {avg_loss:.3f}')
    
    return model

In [ ]:
# Hiperparâmetros do modelo
embed_size, num_hiddens, num_layers, dropout = 32, 32, 2, 0.1
batch_size, num_steps = 64, 10
lr, num_epochs = 0.005, 50

# Define o dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Cria o iterador de dados e o vocabulário
train_iter, src_vocab, tgt_vocab = load_data_nmt(batch_size, num_steps)

# Cria o codificador, decodificador e o modelo completo
encoder = Seq2SeqEncoder(len(src_vocab), embed_size, num_hiddens, num_layers, dropout)
decoder = Seq2SeqDecoder(len(tgt_vocab), embed_size, num_hiddens, num_layers, dropout)
model = EncoderDecoder(encoder, decoder)

# Treina o modelo
train_seq2seq(model, train_iter, lr, num_epochs, device)

### Treinando o Modelo

Agora vamos definir a função de perda e o procedimento de treinamento.

In [ ]:
def predict_seq2seq(model, src_sentence, src_vocab, tgt_vocab, num_steps, device):
    """Prevê para sequência para sequência."""
    # Define o modelo no modo de avaliação
    model.eval()
    
    # Processa a frase de entrada
    src_tokens = src_sentence.lower().split(' ')
    src_tokens = ['<bos>'] + src_tokens + ['<eos>']
    
    # Converte tokens para índices
    src_indices = [src_vocab[token] for token in src_tokens]
    
    # Preenche até o tamanho requerido
    if len(src_indices) < num_steps:
        src_indices += [src_vocab['<pad>']] * (num_steps - len(src_indices))
    else:
        src_indices = src_indices[:num_steps]
    
    # Converte para tensor e adiciona dimensão de lote
    enc_X = torch.tensor(src_indices, dtype=torch.long, device=device).unsqueeze(0)
    
    # Obtém saídas do codificador e inicializa o estado do decodificador
    enc_outputs = model.encoder(enc_X)
    dec_state = model.decoder.init_state(enc_outputs)
    
    # Inicializa a entrada do decodificador com token <bos>
    dec_X = torch.tensor([[tgt_vocab['<bos>']]], dtype=torch.long, device=device)
    
    # Gera tradução
    output_tokens = []
    for _ in range(num_steps):
        Y, dec_state = model.decoder(dec_X, dec_state)
        # Obtém o token com a maior previsão
        dec_X = Y.argmax(dim=2)
        pred_token = tgt_vocab.idx_to_token[dec_X.squeeze(0).item()]
        
        # Para se previmos <eos> ou <pad>
        if pred_token in ['<eos>', '<pad>']:
            break
        output_tokens.append(pred_token)
        
    return ' '.join(output_tokens)

def translate(model, src_sentence, src_vocab, tgt_vocab, num_steps, device):
    """Traduz uma frase da origem para o destino."""
    translation = predict_seq2seq(model, src_sentence, src_vocab, tgt_vocab,
                               num_steps, device)
    print(f'Origem: {src_sentence}')
    print(f'Tradução: {translation}')
    return translation

Agora vamos definir a função de treinamento.

In [ ]:
# Frases em inglês de exemplo para traduzir
english_sentences = [
    'go .',
    'i am hungry .',
    'he is running .'
]

# Traduz cada frase
for sentence in english_sentences:
    translate(model, sentence, src_vocab, tgt_vocab, num_steps, device)

### Criando e Treinando o Modelo Seq2Seq

In [ ]:
embed_size, num_hiddens, num_layers, dropout = 32, 32, 2, 0.1
batch_size, num_steps = 64, 10
lr, num_epochs = 0.005, 300
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iter, src_vocab, tgt_vocab = load_data_nmt(batch_size, num_steps)
encoder = Seq2SeqEncoder(len(src_vocab), embed_size, num_hiddens, num_layers,
                         dropout)
decoder = Seq2SeqDecoder(len(tgt_vocab), embed_size, num_hiddens, num_layers,
                         dropout)
model = EncoderDecoder(encoder, decoder)
train_seq2seq(model, train_iter, lr, num_epochs, device)

### Previsão

In [ ]:
def predict_seq2seq(model, src_sentence, src_vocab, tgt_vocab, num_steps,
                    device, save_attention_weights=False):
    """Prevê para sequência para sequência."""
    # Define o modelo no modo de avaliação para inferência
    model.eval()
    src_tokens = src_vocab[src_sentence.lower().split(' ')]
    src_len = len(src_tokens)
    if src_len < num_steps:
        src_tokens += [src_vocab['<pad>']] * (num_steps - src_len)
    enc_X = torch.unsqueeze(
        torch.tensor(src_tokens, dtype=torch.long, device=device), dim=0)
    enc_outputs = model.encoder(enc_X)
    dec_state = model.decoder.init_state(enc_outputs)
    # Adiciona a dimensão do lote
    dec_X = torch.unsqueeze(
        torch.tensor([tgt_vocab['<bos>']], dtype=torch.long, device=device),
        dim=0)
    output_seq, attention_weight_seq = [], []
    for _ in range(num_steps):
        Y, dec_state = model.decoder(dec_X, dec_state)
        # Usamos o token com a maior probabilidade de previsão como entrada
        # do decodificador no próximo passo de tempo
        dec_X = Y.argmax(dim=2)
        pred = dec_X.squeeze(dim=0).type(torch.int32).item()
        # Uma vez que o token de fim de sequência é previsto, a geração para
        if pred == tgt_vocab['<eos>']:
            break
        output_seq.append(pred)
    return ' '.join(tgt_vocab.to_tokens(output_seq))

In [ ]:
def translate(model, src_sentence, src_vocab, tgt_vocab, num_steps, device):
    """Traduz uma frase da origem para o destino."""
    translation = predict_seq2seq(model, src_sentence, src_vocab, tgt_vocab,
                                  num_steps, device)
    print(f'Origem: {src_sentence}')
    print(f'Tradução: {translation}')
    return translation

Vamos tentar traduzir algumas frases de exemplo do inglês para o francês:

In [ ]:
# Frases de exemplo em inglês
english_sentences = [
    'go .',
    'i am hungry .',
    'he is running .'
]

for sentence in english_sentences:
    translate(model, sentence, src_vocab, tgt_vocab, num_steps, device)

# Discussão

## Análise dos Resultados da Tradução (Questão 1 da Seção 9.5.7)

Com base em nossos experimentos com o modelo de tradução automática, podemos fazer várias observações sobre os resultados da tradução:

1. **Desempenho Básico da Tradução**: O modelo consegue aprender traduções básicas para frases simples. Para frases curtas com vocabulário comum como "go ." ou "i am hungry .", as traduções são geralmente razoáveis.

2. **Limitações de Vocabulário**: Como filtramos palavras que aparecem menos de duas vezes nos dados de treinamento, o vocabulário é limitado. Isso significa que palavras raras provavelmente serão tratadas como tokens desconhecidos, levando à perda de informações na tradução.

3. **Gramática e Contexto**: O modelo às vezes tem dificuldades com precisão gramatical, particularmente com concordância de gênero e conjugações verbais em francês, que dependem do contexto que a arquitetura simples de codificador-decodificador pode não capturar completamente.

4. **Impacto do Tamanho da Sequência**: O desempenho diminui com frases mais longas, pois o modelo tem dificuldade em manter o contexto em sequências mais longas. Esta é uma limitação conhecida dos modelos básicos de sequência-para-sequência sem mecanismos de atenção.

5. **Efeito do Tamanho dos Dados**: Usamos apenas um pequeno subconjunto do conjunto de dados Tatoeba (600 exemplos), o que limita a capacidade do modelo de generalizar para uma ampla variedade de frases.

## Por que Usar a Arquitetura Sequência-para-Sequência para Tradução Automática (Questão 2 da Seção 9.5.7)

As arquiteturas de sequência-para-sequência (seq2seq) são particularmente adequadas para tarefas de tradução automática por várias razões importantes:

1. **Tratamento de Comprimento Variável**: A tradução automática requer mapeamento entre sequências de diferentes comprimentos - frases de origem e suas traduções raramente têm o mesmo número de palavras. Os modelos seq2seq lidam naturalmente com essa exigência de entrada e saída de comprimento variável.

2. **Preservação de Dependências Sequenciais**: Ambos os idiomas têm dependências sequenciais onde a ordem das palavras e o contexto importam. A arquitetura codificador-decodificador captura essas dependências tanto no idioma de origem quanto no de destino.

3. **Preservação de Contexto**: O codificador comprime toda a frase de origem em um vetor de contexto (ou uma série de estados ocultos) que encapsula o significado da sequência de entrada. Isso permite que o decodificador gere uma tradução que considera o significado de toda a frase de origem.

4. **Aprendizado Fim-a-Fim**: Os modelos seq2seq aprendem o mapeamento de tradução diretamente de corpora paralelos sem exigir regras linguísticas explícitas, o que é valioso dada a complexidade da tradução de idiomas.

5. **Flexibilidade Arquitetônica**: A estrutura seq2seq permite vários aprimoramentos, como mecanismos de atenção, que ajudam a resolver o gargalo de informações no vetor de contexto e melhoram significativamente a qualidade da tradução para frases mais longas.